<a href="https://colab.research.google.com/github/Nikhileswar-Komati/Suicide_Ideation/blob/master/S_VS_T_USE_LSTM_LSTM_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import nltk
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import os, re
import tensorflow_hub as hub
import tensorflow as tf
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
os.environ['KAGGLE_USERNAME'] = "nikhileswarkomati"
os.environ['KAGGLE_KEY'] = "001b3a30170775e55950edb6ff0c9b17"
!kaggle datasets download -d nikhileswarkomati/suicide-watch

100% 115M/115M [00:01<00:00, 131MB/s] 
100% 115M/115M [00:01<00:00, 106MB/s]


In [ ]:
!unzip '/content/suicide-watch.zip'

Archive:  /content/suicide-watch.zip
  inflating: SuicideAndDepression_Detection.csv  


In [ ]:
data = pd.read_csv('/content/SuicideAndDepression_Detection.csv', lineterminator = '\n')
data.sample(5)

,text,class
242456,I'm scared.It's not fair. Everyone expects me ...,depression
274632,"I woke up from my ""overdose""Maybe i'll just ad...",SuicideWatch
147145,I’m gonna wreck my shit right here and now Bac...,teenagers
284000,"This hurtsI'm tired, but I have to be alert al...",depression
199500,"In this fake scenario, one has to die. Your mo...",teenagers


In [ ]:
print(data.shape)
data['class'].value_counts()

(348111, 2)


depression      116037
SuicideWatch    116037
teenagers       116037
Name: class, dtype: int64

In [ ]:
# X = data.iloc[:, 0].values
# y = data.iloc[:, 1].values

# le = LabelEncoder()

# y = le.fit_transform(y)

# train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
# train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
# print(train_X.shape, test_y.shape, val_X.shape)

#------------------------------------------------------------------------------------------
X = data.loc[(data['class'] != 'depression'), 'text'].values
y = data.loc[(data['class'] != 'depression'), 'class'].values

le = LabelEncoder()

y = le.fit_transform(y)

train_X, test_X, train_y, test_y = train_test_split(X, y, train_size = 0.5, stratify = y)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, train_size = 0.2, stratify = train_y)
print(train_X.shape, test_y.shape, val_X.shape)

(23207,) (116037,) (92830,)


In [ ]:
embeddings = embed(data.loc[(data['class'] != 'depression'), 'text'][:10000].values)
train_y = le.fit_transform(data.loc[(data['class'] != 'depression'), 'class'][:10000].values)
test_embeddings = embed(data.loc[(data['class'] != 'depression'), 'text'][10000:15000].values)
test_y = le.transform(data.loc[(data['class'] != 'depression'), 'class'][10000:15000].values)


# ------------------------------------------------------------------------------------
# embeddings = embed([preprocess(ele) for ele in data.loc[(data['class'] != 'depression'), 'text'][:10000].values])
# train_y = le.fit_transform(data.loc[(data['class'] != 'depression'), 'class'][:10000].values)
# test_embeddings = embed([preprocess(ele) for ele in data.loc[(data['class'] != 'depression'), 'text'][10000:15000].values])
# test_y = le.transform(data.loc[(data['class'] != 'depression'), 'class'][10000:15000].values)

In [ ]:
embeddings.shape

TensorShape([10000, 512])

In [ ]:
from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import itertools
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.preprocessing import text, sequence
from keras import utils
X_train = embeddings
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

model = Sequential()
# model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2, input_shape = (X_train.shape[1], 1)))
# model.add(Dense(1, activation = 'sigmoid'))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape = (512, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100, dropout = 0.1, recurrent_dropout = 0.1))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
checkpoint = tf.keras.callbacks.ModelCheckpoint('model_lstm.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
callbacks_list = [checkpoint, earlystopping]
model.fit(X_train, train_y, epochs=20, batch_size = 32,verbose = 1, callbacks = callbacks_list, validation_split = 0.1)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 512, 32)           128       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 256, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 53,429
Trainable params: 53,429
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
282/282 [==============================] - 135s 470ms/step - loss: 0.6168 - accuracy: 0.6673 - val_loss: 0.4287 - val_accuracy: 0.8110

Epoch 00001: val_accuracy improved from -inf to 0.81100, saving model to model_lstm.h5
Epoch 2/2

In [ ]:
X_test = test_embeddings
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
from keras.models import load_model
model = load_model('/content/model_lstm.h5')
y_pred = model.predict(X_test)
y_pred = np.rint(y_pred)
y_test = test_y
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.8318
              precision    recall  f1-score   support

         0.0       0.79      0.86      0.82      2293
         1.0       0.88      0.80      0.84      2707

    accuracy                           0.83      5000
   macro avg       0.83      0.83      0.83      5000
weighted avg       0.84      0.83      0.83      5000



In [ ]:
from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
import itertools
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Embedding
from keras.preprocessing import text, sequence
from keras import utils
X_train = embeddings
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

model = Sequential()
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2, input_shape = (512, 1)))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)
callbacks_
list = [checkpoint, earlystopping]
model.fit(X_train, train_y, epochs=20, batch_size = 32,verbose = 1, callbacks = callbacks_list, validation_split = 0.1)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 101       
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
282/282 [==============================] - 219s 765ms/step - loss: 0.6776 - accuracy: 0.5844 - val_loss: 0.6326 - val_accuracy: 0.7240

Epoch 00001: val_accuracy improved from -inf to 0.72400, saving model to model.h5
Epoch 2/20
282/282 [==============================] - 220s 779ms/step - loss: 0.5800 - accuracy: 0.7392 - val_loss: 0.4757 - val_accuracy: 0.8030

Epoch 00002: val_accuracy improved from 0.72400 to 0.80300, saving model to model.h5
Epoch 3/20
282/282 [==========================

In [ ]:
X_test = test_embeddings
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
from keras.models import load_model
model = load_model('/content/model.h5')
y_pred = model.predict(X_test)
y_pred = np.rint(y_pred)
y_test = test_y
print(accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))

0.815
              precision    recall  f1-score   support

         0.0       0.84      0.80      0.82      2613
         1.0       0.79      0.83      0.81      2387

    accuracy                           0.81      5000
   macro avg       0.81      0.82      0.81      5000
weighted avg       0.82      0.81      0.82      5000

